# Chapter 2: Routing

Key Takeaways:
- Routing enables agents to make dynamic decisions about the next step in a workflow based on conditions.
- It allows agents to handle diverse inputs and adapt their behavior, moving beyond linear execution.
- Routing logic can be implemented using LLMs, rule-based systems, or embedding similarity.
- Frameworks like LangGraph and Google ADK provide structured ways to define and manage routing within agent workflows, albeit with different architectural approaches

Routing Patterns:

1. **LLM-based Routing** - Use an LLM to analyze input and decide the next action
2. **Embedding-based Routing** - Route based on semantic similarity using embeddings
3. **Rule-based Routing** - Use conditional logic and pattern matching
4. **Machine Learning Model-Based Routing** - Use trained ML models for classification

### Heuristic: *required control of execution workflow = custom routing*

## Setup and Initialization

In [ ]:
# Copyright (c) 2025 Marco Fago
# https://www.linkedin.com/in/marco-fago/
#
# This code is licensed under the MIT License.
# See the LICENSE file in the repository for the full license text.

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

In [ ]:
# --- Configuration ---
# Ensure your API key environment variable is set (e.g., GOOGLE_API_KEY)
from dotenv import load_dotenv
load_dotenv()

try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
    print(f"Language model initialized: {llm.model}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None

## Define Simulated Sub-Agent Handlers

These functions simulate specialized agents that handle different types of requests. In a real system, these would be separate agents with their own logic and capabilities.

In [ ]:
def booking_handler(request: str) -> str:
    """Simulates the Booking Agent handling a request."""
    print("\n--- DELEGATING TO BOOKING HANDLER ---")
    return f"Booking Handler processed request: '{request}'. Result: Simulated booking action."

In [ ]:
def info_handler(request: str) -> str:
    """Simulates the Info Agent handling a request."""
    print("\n--- DELEGATING TO INFO HANDLER ---")
    return f"Info Handler processed request: '{request}'. Result: Simulated information retrieval."

In [ ]:
def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    print("\n--- HANDLING UNCLEAR REQUEST ---")
    return f"Coordinator could not delegate request: '{request}'. Please clarify."

## Define Coordinator Router Chain

This chain uses an LLM to analyze the user's request and decide which handler should process it. This is the "LLM-based Routing" pattern.

In [ ]:
coordinator_router_prompt = ChatPromptTemplate.from_messages([
    ("system", """Analyze the user's request and determine which specialist handler should process it.
    
- If the request is related to booking flights or hotels, output 'booker'.
- For all other general information questions, output 'info'.
- If the request is unclear or doesn't fit either category, output 'unclear'.

ONLY output one word: 'booker', 'info', or 'unclear'."""),
    ("user", "{request}")
])

In [ ]:
if llm:
    coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()
else:
    coordinator_router_chain = None

## Define the Delegation Logic

We use `RunnableBranch` to route based on the router chain's output. This creates a decision tree that directs requests to the appropriate handler.

In [ ]:
# Define the branches for the RunnableBranch
branches = {
    "booker": RunnablePassthrough.assign(
        output=lambda x: booking_handler(x['request']['request'])
    ),
    "info": RunnablePassthrough.assign(
        output=lambda x: info_handler(x['request']['request'])
    ),
    "unclear": RunnablePassthrough.assign(
        output=lambda x: unclear_handler(x['request']['request'])
    ),
}

In [ ]:
# Create the RunnableBranch. It takes the output of the router chain
# and routes the original input ('request') to the corresponding handler.

delegation_branch = RunnableBranch(
    (lambda x: x['decision'].strip() == 'booker', branches["booker"]),
    (lambda x: x['decision'].strip() == 'info', branches["info"]),
    branches["unclear"]  # Default branch for 'unclear' or any other output
)

## Combine Router and Delegation

Create the complete coordinator agent by combining:
1. The router chain (decides which handler)
2. The delegation branch (routes to the handler)
3. Output extraction (returns the final result)

In [ ]:
if coordinator_router_chain:
    coordinator_agent = (
        {
            "decision": coordinator_router_chain,
            "request": RunnablePassthrough()
        }
        | delegation_branch
        | (lambda x: x['output'])  # Extract the final output
    )
else:
    coordinator_agent = None
    print("Coordinator agent not initialized due to LLM initialization failure.")

## Example Usage

Let's test the routing system with different types of requests to see how it delegates to the appropriate handlers.

In [ ]:
# Example 1: Booking request
if coordinator_agent:
    print("--- Running with a booking request ---")
    request_a = "Book me a flight to London."
    result_a = coordinator_agent.invoke({"request": request_a})
    print(f"Final Result A: {result_a}")
else:
    print("Skipping execution due to LLM initialization failure.")

In [ ]:
# Example 2: Information request
if coordinator_agent:
    print("\n--- Running with an info request ---")
    request_b = "What is the capital of Italy?"
    result_b = coordinator_agent.invoke({"request": request_b})
    print(f"Final Result B: {result_b}")
else:
    print("Skipping execution due to LLM initialization failure.")

In [ ]:
# Example 3: Unclear/complex request
if coordinator_agent:
    print("\n--- Running with an unclear request ---")
    request_c = "Tell me about quantum physics."
    result_c = coordinator_agent.invoke({"request": request_c})
    print(f"Final Result C: {result_c}")
else:
    print("Skipping execution due to LLM initialization failure.")

## Conclusion

This notebook demonstrated **LLM-based Routing** using `RunnableBranch` in LangChain:

1. **Router Chain** - LLM analyzes the request and outputs a decision ('booker', 'info', or 'unclear')
2. **Delegation Branch** - Routes the request to the appropriate handler based on the decision
3. **Handler Functions** - Specialized functions that process specific types of requests

Key benefits:
- Dynamic routing based on natural language understanding
- Flexible and extensible (easy to add new handlers)
- Clear separation of concerns between routing logic and handler implementation